# The three levels of aggregation

In the previous section, the groupby object only finishes the first step of split-apply-combine paradigm. The complexity of implementing the rest may vary. Here's the three situations from simple to complex.

1. If we intend to calculate mean, max, or size, one may just invoke them on the groupby object. It's part of groupby object and has been optimized.
2. If the method exists for each dataframe chunk, like quantile, but is not part of groupby object, we can still just call it. Internally, groupby slices up the series/dataframes and calls quantile function for each piece, and stiches them together into the result. 
3. If we want a customized method that is neither part of groupby nor part of dataframe/series, we got to write our own aggregation function before pass it to the agg method.

There is technical differences between the first two situations but to the end user, they should look the same.

In [1]:
import pandas as pd 
tips = pd.read_csv('tips.csv')
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [2]:
example_of_scenario_1 = tips.groupby(['day']).mean()[['total_bill','tip']]
example_of_scenario_1

,total_bill,tip
day,,
Fri,17.151579,2.734737
Sat,20.441379,2.993103
Sun,21.410000,3.255132
Thur,17.682742,2.771452


In [3]:
example_of_scenario_2 = tips.groupby(['day']).quantile(0.9)[['total_bill','tip']]
example_of_scenario_2.columns.names =['']
example_of_scenario_2

,total_bill,tip
day,,
Fri,27.618,4.060
Sat,31.894,4.802
Sun,33.765,5.035
Thur,28.316,4.920


In [11]:
def find_range(arr):
    return arr.max() - arr.min()
example_of_scenario_3 = tips.groupby(['day']).agg(find_range)[['total_bill','tip']]
example_of_scenario_3.head()

,total_bill,tip
day,,
Fri,34.42,3.73
Sat,47.74,9.00
Sun,40.92,5.49
Thur,35.60,5.45


# More operations and more variables

This section settles two major questions:
1. what if there is more than one operation we desire to impose for one variable
2. what if for different variable we want to define a different operations

In [6]:
import numpy as np 
solution_to_question_1 = tips.groupby(['day']).agg(['std', np.mean, find_range])[['total_bill', 'tip']]
solution_to_question_1

total_bill                             tip                     
            std       mean find_range       std      mean find_range
day                                                                 
Fri    8.302660  17.151579      34.42  1.019577  2.734737       3.73
Sat    9.480419  20.441379      47.74  1.631014  2.993103       9.00
Sun    8.832122  21.410000      40.92  1.234880  3.255132       5.49
Thur   7.886170  17.682742      35.60  1.240223  2.771452       5.45

In [12]:
solution_to_question_1_custname = tips.groupby(['day']).agg([('standard dev', np.std), ('mean',np.mean), ('range',find_range)])[['total_bill', 'tip']]
solution_to_question_1_custname

total_bill                            tip                
     standard dev       mean  range standard dev      mean range
day                                                             
Fri      8.302660  17.151579  34.42     1.019577  2.734737  3.73
Sat      9.480419  20.441379  47.74     1.631014  2.993103  9.00
Sun      8.832122  21.410000  40.92     1.234880  3.255132  5.49
Thur     7.886170  17.682742  35.60     1.240223  2.771452  5.45

To recap, we simply need to pass a list of operations into the agg method. It allows a mixture of built-in functions (you can call by giving a string ('mean', 'std' for instance)), numpy aggregation functions and even customized funtions. 

Moreover, one can give the function a name by passing in a tuple. The first arg of the tuple is name the second is the operation (function). The point of this is to make the resulting columns more readable. To be more specific, if there is only one operation for one column, the column name will remain the same, but as long as there is more than one operation, the column names in the output will become hierarchical and multi-indexing comes into effect. This means the original name becomes level 0 (top level) and the columns generaetd by given operations are going to have level 1 names.

Note that function used here has to be aggregation function, which produces scalar values from arrays. 

In [8]:
case_2_2 = tips.groupby(['day', 'smoker']).agg({'total_bill': [('max_value',np.max)], 'tip': [('range',find_range),('mean', 'mean')]})

In [9]:
case_2_2

total_bill   tip          
             max_value range      mean
day  smoker                           
Fri  No          22.75  2.00  2.812500
     Yes         40.17  3.73  2.714000
Sat  No          48.33  8.00  3.102889
     Yes         50.81  9.00  2.875476
Sun  No          48.17  4.99  3.167895
     Yes         45.35  5.00  3.516842
Thur No          41.19  5.45  2.673778
     Yes         43.11  3.00  3.030000

The answer is pass in a dictionary. The keys are names of columns to be aggregated, and the values are operations. If there is more than one operation, simply pass a list; if there is a need to customize the name of column, pass a tuple instead.